In [24]:
import pymongo
import pandas as pd
import matplotlib as plt
from pandas import Series,DataFrame
pd.set_option('display.width', None)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
pd.set_option('display.max_columns', None)  # 设置显示最大行
#二行即可搞定画图中文乱码
plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus']=False #用来正常显示负号
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#连接数据库
client = pymongo.MongoClient('localhost',27017)
futures = client.futures

market = futures.market
unit = futures.unit
position = futures.position

#加载数据
market = DataFrame(list(market.find()))
unit = DataFrame(list(unit.find()))
position = DataFrame(list(position.find()))

#大写字母
position['variety']=position['variety'].str.upper()

#删除/选取某行含有特殊数值的列
position=position.set_index('名次')

#选择需要显示的字段
data1=market[['date','variety','set_close']]

data2=unit[[ 'variety','unit']]
position=position[['date','variety','symbol','持买仓量期货公司','持买仓量', '持买仓量增减','持卖仓量期货公司','持卖仓量', '持卖仓量增减']]

#查询会员
members='中粮期货'
data3=position[(position['持买仓量期货公司'] == members)]
#汇总合约
data3=data3[['date','variety','持买仓量期货公司','持买仓量']]
data3=data3.groupby(['date','variety','持买仓量期货公司'])[['持买仓量']].sum()
data4=position[(position['持卖仓量期货公司'] == members)]
data4=data4[['date','variety','持卖仓量期货公司','持卖仓量']]
data4=data4.groupby(['date','variety','持卖仓量期货公司'])[['持卖仓量']].sum()
print(data3.head())
print(data4.head())

#并集
data5=pd.merge(data3,data4, on=['date','variety'],how='outer')
data5['会员简称']=data5.apply(lambda x: members,axis=1)

#nan缺失值填充fillna()为0
data5=data5.fillna(0)
data5['净持仓']=data5.apply(lambda x: x['持买仓量']-x['持卖仓量'],axis=1)
#选择需要显示的字段
data5=data5[['会员简称','持买仓量','持卖仓量','净持仓']]
data5=data5.reset_index(['variety','date'])
print(data5.head())
netpostion=data5.set_index('date')

netpostion=data5.pivot_table('净持仓',index='date',columns='variety',fill_value=0)

#合约价值
contractValue=pd.merge(data1,data2,how='left',sort=False).drop_duplicates()
contractValue['contractValue'] = contractValue.apply(lambda x: x['set_close']*x['unit'],axis=1)
contractValue=contractValue[['date','variety','contractValue']]
#值替换replace()
contractValue=contractValue.replace(['TA'],'PTA')
print(contractValue.head())
sz=pd.merge(data5,contractValue,on=['date','variety'],how='left')

#净持仓价值
sz['净持仓价值']=sz.apply(lambda x: x['净持仓']*x['contractValue'],axis=1)
sz=sz[['date','variety','会员简称','净持仓价值']]
sz=sz.sort_values(by='净持仓价值')
sz=sz.pivot_table('净持仓价值',index=['date','会员简称'],columns='variety',fill_value=0)
sz=sz.copy()

sz['化工板块']=sz[['RU','MA','V','L','PP','BU','PTA']].sum(axis=1)
sz['油脂板块']=sz[['Y','P','OI']].sum(axis=1)
sz['有色板块']=sz[['CU','AL','ZN','NI','PB']].sum(axis=1)
sz['黑色板块']=sz[['RB','HC','J','JM','I']].sum(axis=1)
sz['黄金白银']=sz[['AU','AG']].sum(axis=1)
sz['工业品']=sz[['CU','AL','ZN','NI','PB','RB','FG','RU','L','V','PTA','MA','PP','JM','J','I','BU','HC','FU']].sum(axis=1)#ZC
sz['农产品']=sz[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP']].sum(axis=1)
sz['商品板块']=sz.sum(axis=1)
sz


chemical=sz.copy()
chemical=chemical[['RU','MA','V','L','PP','BU','PTA','商品板块','工业品','化工板块']]
chemical['max']=chemical[['RU','MA','V','L','PP','BU','PTA']].idxmax(axis=1)
chemical['min']=chemical[['RU','MA','V','L','PP','BU']].idxmin(axis=1)
chemical


                                持买仓量
date       variety 持买仓量期货公司         
2018-10-08 A       中粮期货        763.0
           AG      中粮期货       4050.0
           AL      中粮期货      15496.0
           BU      中粮期货       4358.0
           C       中粮期货      54026.0
                                持卖仓量
date       variety 持卖仓量期货公司         
2018-10-08 A       中粮期货       2643.0
           AL      中粮期货      18253.0
           AU      中粮期货        593.0
           BU      中粮期货       8040.0
           C       中粮期货      49654.0
         date variety  会员简称     持买仓量     持卖仓量     净持仓
0  2018-10-08       A  中粮期货    763.0   2643.0 -1880.0
1  2018-10-08      AG  中粮期货   4050.0      0.0  4050.0
2  2018-10-08      AL  中粮期货  15496.0  18253.0 -2757.0
3  2018-10-08      BU  中粮期货   4358.0   8040.0 -3682.0
4  2018-10-08       C  中粮期货  54026.0  49654.0  4372.0
         date variety  contractValue
0  2018-10-08      AG        52965.0
1  2018-10-08      AL        72395.0
2  2018-10-08      AP       117160.0
3  2018-10

,variety,RU,MA,V,L,PP,BU,PTA,商品板块,工业品,化工板块,max,min
date,会员简称,,,,,,,,,,,,
2018-10-08,中粮期货,-546440500,0,-227373810,0,38953600,-138664120,-171458430,-71044615790,-5477522010,-1044983260,PP,RU
2018-10-09,中粮期货,-602125050,0,-232662000,-140993220,10245140,-152837440,-40563280,-76687351005,-5665513155,-1158935850,PP,RU
2018-10-10,中粮期货,-624699400,0,-91456305,-216275465,-27529640,-34739650,43456875,-78571501110,-5921287420,-951243585,PTA,RU
2018-10-11,中粮期货,-579458880,0,-49541940,-194493285,-65945940,-70439160,280659280,-76653224215,-5683207065,-679219925,PTA,RU
2018-10-12,中粮期货,-573701805,0,0,-191917215,-75050430,-74107760,310544370,-74116812685,-4841636035,-604232840,PTA,RU
2018-10-15,中粮期货,-560791225,0,-85019880,-146206315,-299395320,-67025300,394971610,-72862043073,-4913318191,-763466430,PTA,RU
2018-10-16,中粮期货,-558753860,0,-99848060,-128613510,-389431810,-172420820,366955150,-70711991125,-4920329935,-982112910,PTA,RU
2018-10-17,中粮期货,-558930700,0,-99612500,-124307745,-459659100,-62678480,328660345,-71195885948,-5196281486,-976528180,PTA,RU
2018-10-18,中粮期货,-539262875,0,-162318245,-86886580,-519282000,-44699960,307195720,-69060480607,-4987150599,-1045253940,PTA,RU


In [12]:
黑色板块=sz.copy()
黑色板块=黑色板块[['RB','HC','J','JM','I','黑色板块','工业品','商品板块']] 
黑色板块['max']=黑色板块[['RB','HC','J','JM','I']].idxmax(axis=1) 
黑色板块['min']=黑色板块[['RB','HC','J','JM','I']].idxmin(axis=1)
黑色板块

,variety,RB,HC,J,JM,I,黑色板块,工业品,商品板块,max,min
date,会员简称,,,,,,,,,,
2018-10-08,中粮期货,-857700360,3491100,-623042000,185395500,197604000,-1094251760,-5477522010,-71044615790,I,RB
2018-10-09,中粮期货,-740564400,3537900,0,0,271856900,-465169600,-5665513155,-76687351005,I,RB
2018-10-10,中粮期货,-859542330,3538800,0,-165156120,296485500,-724674150,-5921287420,-78571501110,I,RB
2018-10-11,中粮期货,-687155070,3504600,0,-193365900,283750400,-593265970,-5683207065,-76653224215,I,RB
2018-10-12,中粮期货,0,0,0,-190956840,349383600,158426760,-4841636035,-74116812685,I,JM
2018-10-15,中粮期货,0,0,0,-324328800,358581000,34252200,-4913318191,-72862043073,I,JM
2018-10-16,中粮期货,0,0,0,-338003040,82916100,-255086940,-4920329935,-70711991125,I,JM
2018-10-17,中粮期货,0,0,0,-321092640,44455000,-276637640,-5196281486,-71195885948,I,JM
2018-10-18,中粮期货,-74079180,0,0,-322543260,-37689300,-434311740,-4987150599,-69060480607,HC,JM


In [14]:
oil=sz.copy()
oil=oil[['Y','P','OI','油脂板块','农产品']]
oil['max']=oil[['Y','P','OI']].idxmax(axis=1) 
oil['min']=oil[['Y','P','OI']].idxmin(axis=1)
oil['2018-10-21':]

,variety,Y,P,OI,油脂板块,农产品,max,min
date,会员简称,,,,,,,
2018-10-22,中粮期货,-5943453090,-758963520,-2052467060,-8754883670,-22564072605,P,Y
2018-10-23,中粮期货,-5908436730,-790305750,-1996948080,-8695690560,-22536963625,P,Y
2018-10-24,中粮期货,-5577101200,-685301790,-1843272800,-8105675790,-22045073530,P,Y
2018-10-25,中粮期货,-5374232960,-862337280,-1757536500,-7994106740,-21283744840,P,Y
2018-10-26,中粮期货,-5312536810,-1034237620,-1597992000,-7944766430,-21407938400,P,Y
2018-10-29,中粮期货,-5374043100,-974394480,-1642553280,-7990990860,-20676009510,P,Y
2018-10-30,中粮期货,-5360676200,-913192380,-1652571600,-7926440180,-19634834450,P,Y
2018-10-31,中粮期货,-5045313520,-912230810,-1640930320,-7598474650,-19239356285,P,Y
2018-11-01,中粮期货,-4998007080,-931916960,-1624006920,-7553930960,-18151894080,P,Y


In [16]:
有色=sz.copy()
有色=有色[['CU','AL','ZN','NI','PB','有色板块','工业品','商品板块']]
有色['max']=有色[['CU','AL','ZN','NI','PB']].idxmax(axis=1) 
有色['min']=有色[['CU','AL','ZN','NI','PB']].idxmin(axis=1)
有色

,variety,CU,AL,ZN,NI,PB,有色板块,工业品,商品板块,max,min
date,会员简称,,,,,,,,,,
2018-10-08,中粮期货,-1883887680,-199593015,-449287560,-760855860,7669125,-3285954990,-5477522010,-71044615790,PB,CU
2018-10-09,中粮期货,-2021899060,-167662365,-1059156000,-737042165,33713685,-3952045905,-5665513155,-76687351005,PB,CU
2018-10-10,中粮期货,-2125892930,-223968095,-1355685090,-490269970,33540500,-4162275585,-5921287420,-78571501110,PB,CU
2018-10-11,中粮期货,-2230454720,-132418560,-1450356390,-613586480,72163140,-4354653010,-5683207065,-76653224215,PB,CU
2018-10-12,中粮期货,-2255304500,-158495060,-1553612500,-638110585,218937810,-4386584835,-4841636035,-74116812685,PB,CU
2018-10-15,中粮期货,-2145604505,-141344690,-1532937040,-532164926,185444480,-4166606681,-4913318191,-72862043073,PB,CU
2018-10-16,中粮期货,-2043577025,-10475220,-1215159920,-568863900,184583660,-3653492405,-4920329935,-70711991125,PB,CU
2018-10-17,中粮期货,-2052721360,-48122865,-1459126305,-546658416,183929600,-3922699346,-5196281486,-71195885948,PB,CU
2018-10-18,中粮期货,-2066710620,271584000,-1401753750,-472309734,181913655,-3487276449,-4987150599,-69060480607,AL,CU


In [18]:
农产品=sz.copy()
农产品=农产品[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP','农产品']]
农产品['max']=农产品[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP']].idxmax(axis=1) 
农产品['min']=农产品[['A','C','M','RM','Y','P','OI','CF','SR','JD','CS','AP']].idxmin(axis=1)
农产品

,variety,A,C,M,RM,Y,P,OI,CF,SR,JD,CS,AP,农产品,max,min
date,会员简称,,,,,,,,,,,,,,,
2018-10-08,中粮期货,-72135600,83068000,-10571216880,-473522700,-5884184600,-1187532320,-1927656600,-951577980,-1722631040,-12190970,-167657980,0,-22887238670,C,M
2018-10-09,中粮期货,-70988430,37812610,-11708944240,-465689670,-6037045200,-1222819850,-2113790800,-1339175145,-1889234360,-66094400,-167276100,0,-25043245585,C,M
2018-10-10,中粮期货,-72674730,-51712500,-12080507280,-452070060,-6191046800,-1267950530,-2088711440,-1281587500,-1781153640,-71527550,-183702480,0,-25522644510,AP,M
2018-10-11,中粮期货,-68888600,-95108780,-12293135260,-378339300,-5995848150,-1086998610,-2042354160,-1233188160,-1696852080,-12608490,-199487750,0,-25102809340,AP,M
2018-10-12,中粮期货,15505520,-68229000,-11911564760,-389542250,-6183597280,-1153747160,-2120163920,-1045172750,-1769439700,-12648400,-265600550,0,-24904200250,A,M
2018-10-15,中粮期货,0,-6687060,-11381516080,-380970780,-6172896560,-1173118240,-2118392760,-935667530,-1734385100,0,-264966000,0,-24168600110,A,M
2018-10-16,中粮期货,12867660,35551040,-10796805880,-348648300,-6140926260,-1182305280,-2098720680,-751912720,-1755072480,0,-250284850,0,-23276257750,C,M
2018-10-17,中粮期货,13621900,59730440,-10618470030,-351293640,-6107730600,-1200838800,-2093566080,-813406155,-1730548340,0,-265743410,0,-23108244715,C,M
2018-10-18,中粮期货,12635220,124270560,-10528849040,-357126880,-5833562880,-1083398680,-2071118910,-661943565,-1898712080,0,-264251400,0,-22562057655,C,M


In [19]:
板块=sz.copy()
板块=板块[['商品板块','工业品','化工板块','黑色板块','有色板块','农产品','油脂板块']]
板块['2018-10-1':]

,variety,商品板块,工业品,化工板块,黑色板块,有色板块,农产品,油脂板块
date,会员简称,,,,,,,
2018-10-10,中粮期货,-78571501110,-5921287420,-951243585,-724674150,-4162275585,-25522644510,-9547708770
2018-10-11,中粮期货,-76653224215,-5683207065,-679219925,-593265970,-4354653010,-25102809340,-9125200920
2018-10-12,中粮期货,-74116812685,-4841636035,-604232840,158426760,-4386584835,-24904200250,-9457508360
2018-10-15,中粮期货,-72862043073,-4913318191,-763466430,34252200,-4166606681,-24168600110,-9464407560
2018-10-16,中粮期货,-70711991125,-4920329935,-982112910,-255086940,-3653492405,-23276257750,-9421952220
2018-10-17,中粮期货,-71195885948,-5196281486,-976528180,-276637640,-3922699346,-23108244715,-9402135480
2018-10-18,中粮期货,-69060480607,-4987150599,-1045253940,-434311740,-3487276449,-22562057655,-8988080470
2018-10-19,中粮期货,-71437517578,-5439907186,-1150000650,-522498900,-3744808156,-23202463235,-8727808830
2018-10-22,中粮期货,-69276562220,-5176415720,-1093355680,-383095750,-3718779470,-22564072605,-8754883670
